### Imports

In [1]:
from os.path import join

from keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, Callback
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import Adam

import tensorflow as tf

import os
import numpy as np

from batch_generator import BatchGenerator, BatchSequence

from sklearn.metrics import recall_score, precision_score, f1_score

import gzip, pickle

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load data

In [2]:
# Images
images_path_train = os.path.abspath('data/train/')
images_path_validation = os.path.abspath('data/validation/')
images_path_test = os.path.abspath('data/test/')

# Labels
with gzip.open('data/y_train.pickle','rb') as fp:
    y_train = pickle.load(fp)
with gzip.open('data/y_validation.pickle','rb') as fp:
    y_validation = pickle.load(fp)

### Metrics / callbacks

In [3]:
class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.mean_f1s = []
        self.recalls = []
        self.precisions = []

    def on_epoch_end(self, epoch, logs={}):
        y_pred = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        y_true = self.validation_data[1]

        mean_f1 = f1_score(y_true, y_pred, average='micro')
        recall = recall_score(y_true, y_pred, average='micro')
        precision = precision_score(y_true, y_pred, average='micro')
        self.mean_f1s.append(mean_f1)
        self.recalls.append(recall)
        self.precisions.append(precision)

        print('mean_F1: {} — precision: {} — recall: {}'.format(mean_f1, precision, recall))

metrics = Metrics()

In [4]:
# checkpoint
filepath="models/inceptionV3-fc-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

## Network

In [5]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [6]:
batch_size=128*2 # 128 per GPU

In [7]:
# load the generators
training_gen = BatchGenerator(input_dir=images_path_train, y=y_train, batch_size=batch_size)

In [8]:
val_gen = BatchSequence(input_dir=images_path_validation, y=y_validation, batch_size=batch_size)

model       trainable layers
vgg16       4
vgg19       5
exception   6
inception   17
resnet50    7

#### Function that takes modelpath and number of epochs and trains the model at the specified path for the number of epochs. After each epoch the model is saved.

In [9]:
def train_network(modelname, epochs):    
    with tf.device('/cpu:0'):
        model = load_model('models/{}-fc-1_epochs.h5'.format(modelname))

        trainable_layers = layer_dict[modelname[:5]]
        for layer in model.layers[:trainable_layers]:
            layer.trainable = False
        for layer in model.layers[trainable_layers:]:
            layer.trainable = True

    # Multi-GPU data parallelism
    multi_model = multi_gpu_model(model, gpus=2)

    # Use binary loss instead of categorical loss to penalize each output independently, also use lower learning rate
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    multi_model.compile(optimizer=optimizer, loss='binary_crossentropy')

    steps = int(y_train.shape[0]/batch_size) + 1

    #history = model.fit_generator(training_gen, steps_per_epoch=steps, epochs=1, callbacks=[checkpoint, metrics], validation_data=val_gen)
    for i in range(epochs):
        history = multi_model.fit_generator(training_gen, steps_per_epoch=steps, epochs=1)
        model.save("models/{}-finetuned-{}_epochs.h5".format(modelname, i+1))

#### Train all models, if you want to train only 1 model then remove the loop :P

In [10]:
models = ['Xception', 'VGG16', 'VGG19', 'ResNet50']
layer_dict = {'VGG16':-8, 'VGG19':-9, 'Xcept':-10, 'ResNe':-11, 'Incep':-21}
epochs = 1

train_network('VGG19', epochs)
train_network('Xception', epochs)

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/1
1270/3964 [========>.....................] - ETA: 3:24:41 - loss: 0.0606

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,290,290] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: replica_0/model_1/block1_conv2/convolution = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](replica_0/model_1/block1_conv1/Relu, block1_conv2/kernel/read/_205)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training/Adam/gradients/dense_2_1/concat_grad/Slice_1/_313 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:1", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_685_training/Adam/gradients/dense_2_1/concat_grad/Slice_1", tensor_type=DT_FLOAT, _device="/job:localhost/repl

### Test models


In [ ]:
predict_gen = BatchSequence(input_dir=images_path_validation, y=y_validation, batch_size=128)

for path in ['models/VGG19-fc-1_epochs.h5', 'models/Xception-fc-1_epochs.h5', 'models/VGG16-fc-1_epochs.h5', 'models/ResNet50-fc-1_epochs.h5']:
    model = load_model(path)
    # Train only the top few layers
        
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=optimizer, loss='binary_crossentropy') # training configuration
    
    predictions = model.predict_generator(predict_gen, verbose=1)

    y_true = y_validation
    y_pred = (predictions > 0.5).astype(int)

    pr = precision_score(y_true, y_pred, average='micro')
    rc = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    print("[{}] Precision: {} Recall: {} F1: {}".format(path, pr, rc, f1))
    break